In [0]:
from __future__ import print_function
! pip install dropbox
from dropbox import Dropbox
import time
from PIL import Image
import numpy as np

import tensorflow as tf
from tensorflow.contrib.keras import backend
from tensorflow.contrib.keras import models
from tensorflow.contrib.keras import applications

from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave


# style_map
style_map={}
style_map.update({'Painting':'painting.jpg'})
style_map.update({'Wave':'wave.png'})
style_map.update({'Modern Art':'modern_Art.jpg'})

# constants
height = 512
width = 512

# Helper functions and classes
def content_loss(content, combination):
	return backend.sum(backend.square(combination - content))


def gram_matrix(x):
    features = backend.batch_flatten(backend.permute_dimensions(x, (2, 0, 1)))
    gram = backend.dot(features, backend.transpose(features))
    return gram


def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = height * width
    return backend.sum(backend.square(S - C)) / (4. * (channels ** 2) * (size ** 2))


def total_variation_loss(x):
    a = backend.square(x[:, :height-1, :width-1, :] - x[:, 1:, :width-1, :])
    b = backend.square(x[:, :height-1, :width-1, :] - x[:, :height-1, 1:, :])
    return backend.sum(backend.pow(a + b, 1.25))


def eval_loss_and_grads(f_outputs, x):
    x = x.reshape((1, height, width, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    grad_values = outs[1].flatten().astype('float64')
    return loss_value, grad_values

def neural_style_script(content_image, style_image):
  content_image = Image.open(content_image)
  content_image = content_image.resize((width, height))
  style_image = style_map.get(style_image)
  style_image = Image.open(style_image)
  style_image = style_image.resize((width, height))
  content_array = np.asarray(content_image, dtype='float32')
  content_array = np.expand_dims(content_array, axis=0)
  style_array = np.asarray(style_image, dtype='float32')
  style_array = np.expand_dims(style_array, axis=0)
  content_array[:, :, :, 0] -= 103.939
  content_array[:, :, :, 1] -= 116.779
  content_array[:, :, :, 2] -= 123.68
  content_array = content_array[:, :, :, ::-1]
  style_array[:, :, :, 0] -= 103.939
  style_array[:, :, :, 1] -= 116.779
  style_array[:, :, :, 2] -= 123.68
  style_array = style_array[:, :, :, ::-1]
  content_image = backend.variable(content_array)
  style_image = backend.variable(style_array)
  combination_image = backend.placeholder((1, height, width, 3))
  input_tensor = backend.concatenate([content_image,
    style_image,
    combination_image], axis=0)
  model = applications.VGG16(input_tensor=input_tensor, weights='imagenet',include_top=False)  
  content_weight = 0.025
  style_weight = 5.0
  total_variation_weight = 1.0
  loss = backend.variable(0.)
  layers = dict([(layer.name, layer.output) for layer in model.layers])
  layer_features = layers['block2_conv2']
  content_image_features = layer_features[0, :, :, :]
  combination_features = layer_features[2, :, :, :]
  loss = loss + content_weight * content_loss(content_image_features, combination_features)
  feature_layers = ['block1_conv2', 'block2_conv2',
  'block3_conv3', 'block4_conv3',
  'block5_conv3']
  for layer_name in feature_layers:
    layer_features = layers[layer_name]
    style_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_features, combination_features)
    loss += (style_weight / len(feature_layers)) * sl

  loss += total_variation_weight * total_variation_loss(combination_image)
  grads = backend.gradients(loss, combination_image)
  outputs = [loss]
  outputs += grads
  f_outputs = backend.function([combination_image], outputs)
  evaluator = Evaluator(f_outputs)
  x = np.random.uniform(0, 255, (1, height, width, 3)) - 128.
  iterations = 1
  for i in range(iterations):
     print('Start of iteration', i)
     start_time = time.time()
     x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
       fprime=evaluator.grads, maxfun=20)
     print('Current loss value:', min_val)
     end_time = time.time()
     print('Iteration %d completed in %ds' % (i, end_time - start_time))
  x = x.reshape((height, width, 3))
  x = x[:, :, ::-1]
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68
  x = np.clip(x, 0, 255).astype('uint8')
  Image.fromarray(x).save(file)


class Evaluator(object):
    def __init__(self, f_outputs):
        self.loss_value = None
        self.grads_values = None
        self.f_outputs = f_outputs

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(self.f_outputs, x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values    


while(True):
  #API Token
  dbx = Dropbox('56Avmht1CGkAAAAAAAABS8gYYmjrmVC5HZc6ThJbMT168Y1MrWhY0wjfXUln64i4')


  # Iterate over styles folder and download on colab
  for entry in dbx.files_list_folder('/prisma/styles/').entries:
    dbx.files_download_to_file("./"+entry.name, '/prisma/styles/'+entry.name)

  # Iterate over output subfolders
  out_sub_folders=[]
  for entry in dbx.files_list_folder('/prisma/output/').entries:
    out_sub_folders.insert(1,entry.name)

  # Iterate over files in each output subfolder and add it to a set 
  out_files = set()
  for folder in out_sub_folders:  
    for entry in dbx.files_list_folder('/prisma/output/'+folder).entries:
      out_files.add(folder+"/"+entry.name)
  
  in_sub_folders=[]
  to_process_images=[]

  # Iterate over input subfolders 
  for entry in dbx.files_list_folder('/prisma/input/').entries:
    in_sub_folders.insert(1, entry.name)

  # Iterate over each file in input subfolders and check whether in output folder
  # if not add it in to_process_images
  for folder in in_sub_folders:  
    for entry in dbx.files_list_folder('/prisma/input/'+folder).entries:
      if folder+"/"+entry.name not in out_files:
        to_process_images.insert(1,"/prisma/input/"+folder+"/"+entry.name)

  # download from dropbox, files toprocess
  # extract the filename to refer further
  filenames_toproc =[]
  for file in to_process_images:
    tokens= file.split('/')
    filenames_toproc.insert(1,tokens[len(tokens)-1]+"/"+tokens[len(tokens)-2])
    dbx.files_download_to_file("./"+tokens[len(tokens)-1], file)

  for file in filenames_toproc:
    print("1---->",file)
    neural_style_script(file.split("/")[0], file.split("_")[0])
    with open(file.split("/")[0], "rb") as imageFile:
        output = imageFile.read()
    dbx.files_upload(output,'/prisma/output/'+file.split("/")[1]+'/'+file.split("/")[0])




1----> Wave_ak.jpg/ak5146@nyu.edu
Start of iteration 0
